In [18]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import sys
import math
import logging
import yaml
sys.path.append('../')

#import dask_geopandas
# some data need to be converted to multi-type again
from utils.save_gdf_to_gdb import save_gdf_to_gdb
#from process.footprint_report import update_ln, update_poly, update_pt
from shapely import Polygon, MultiPolygon
from shapely import affinity
#import dask_geopandas

from its_logging.logger_config import logger
logger = logging.getLogger('process.footprint')

In [2]:
upload_path = r"D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload"

In [48]:
os.listdir(upload_path)

['ITS_V2.0_Activity.gdb',
 'ITS_V2_0_Footprint_Points.gdb',
 'ITS_V2_0_Footprint_Polygons.gdb',
 'ITS_V2_0_Lines.gdb',
 'ITS_V2_0_Points.gdb',
 'ITS_V2_0_Polygons.gdb']

In [49]:
for f in os.listdir(upload_path):
    gdb_layer = gpd.list_layers(os.path.join(upload_path,f)).loc[0, 'name']
    gdf = gpd.read_file(os.path.join(upload_path,f), driver='OpenFileGDB', layer=gdb_layer)
    gdf.geometry = gdf.make_valid()
    if len(gdf.geom_type.unique()) > 1:
        print("Found multiple geom types")
        break
    save_gdf_to_gdb(gdf,os.path.join(upload_path,f),gdb_layer)

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-27 12:04:21,192 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:04:21,192 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:04:21,203 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Lines.gdb


2025-08-27 12:04:41,099 INFO  [pyogrio._io           ]  Created 131,658 records
2025-08-27 12:04:45,052 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Lines.gdb ITS_V2_0_Lines
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
2025-08-27 12:04:48,145 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:04:48,145 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:04:48,147 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Points.gdb


2025-08-27 12:04:49,100 INFO  [pyogrio._io           ]  Created 20,443 records
2025-08-27 12:04:49,240 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Points.gdb ITS_V2_0_Points
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


Found multiple geom types


In [59]:
gdf.geom_type.unique()

array(['MultiPolygon'], dtype=object)

In [56]:
def get_poly(geometry):
    if isinstance(geometry, Polygon):
        return MultiPolygon([geometry])
    elif isinstance(geometry, MultiPolygon):
        return geometry
    else:
        out = []
        for geom in geometry.geoms:
            if isinstance(geom, Polygon):
                out.append(geom)
            elif isinstance(geom, MultiPolygon):
                for g in geom.geoms:
                    out.append(g)
            else:
                continue
        
        return MultiPolygon(out)
        

In [57]:
gdf.geometry = gdf.geometry.apply(get_poly)

In [58]:
np.all(gdf.is_valid)

np.True_

In [60]:
save_gdf_to_gdb(gdf,os.path.join(upload_path,f),gdb_layer)

2025-08-27 12:12:00,993 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-08-27 12:12:00,993 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-08-27 12:12:01,030 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Polygons.gdb


2025-08-27 12:12:28,509 INFO  [pyogrio._io           ]  Created 454,597 records
2025-08-27 12:12:35,637 INFO  [utils.save_gdf_to_gdb ]        File saved to D:\WORK\wildfire\Interagency-Tracking-System\its\ITSGDB_backup\V2.0upload\ITS_V2_0_Polygons.gdb ITS_V2_0_Polygons


In [52]:
for f in os.listdir(upload_path):
    gdb_layer = gpd.list_layers(os.path.join(upload_path,f)).loc[0, 'name']
    gdf = gpd.read_file(os.path.join(upload_path,f), driver='OpenFileGDB', layer=gdb_layer)
    print(np.all(gdf.is_valid))

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


...

True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


True


C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


False


In [53]:
gdb_layer

'ITS_V2_0_Polygons'

In [54]:
gdf.geometry = gdf.make_valid()

In [55]:
gdf.geom_type.unique()

array(['MultiPolygon', 'GeometryCollection'], dtype=object)

In [2]:
fpath = r"D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb"

In [4]:
gpd.list_layers(fpath)

,name,geometry_type
0,appended_poly,MultiPolygon
1,appended_line,MultiLineString
2,appended_point,Point


In [5]:

#enriched_points = gpd.read_file(os.path.join(upload_path,'ITS_V2_0_Points.gdb'), driver='OpenFileGDB', layer='ITS_V2_0_Points')
enriched_polygons = gpd.read_file(fpath, driver='OpenFileGDB', layer='appended_poly')
enriched_points = gpd.read_file(fpath, driver='OpenFileGDB', layer='appended_point')
enriched_lines = gpd.read_file(fpath, driver='OpenFileGDB', layer='appended_line')

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: organizePolygons() received a polygon with more than 100 parts. The processing may be really slow.  You can skip the processing by setting METHOD=SKIP, or only make it analyze counter-clock wise parts by setting METHOD=ONLY_CCW if you can assume that the outline of holes is counter-clock wise defined
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(
C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\pyogrio\raw.py:198: RuntimeWarning: driver OpenFileGDB does not support open option DRIVER
  return ogr_read(


In [144]:
assert np.all(enriched_points.is_valid)
assert np.all(enriched_lines.is_valid)
assert np.all(enriched_polygons.is_valid)

In [145]:
# Admin org must be populated if counts to mas
assert len(enriched_points[(enriched_points.COUNTS_TO_MAS=="YES") & (enriched_points.ADMINISTERING_ORG.isna())]) == 0
assert len(enriched_lines[(enriched_lines.COUNTS_TO_MAS=="YES") & (enriched_lines.ADMINISTERING_ORG.isna())]) == 0
assert len(enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.ADMINISTERING_ORG.isna())]) == 0

In [146]:
# Agency must be populated if counts to mas
assert len(enriched_points[(enriched_points.COUNTS_TO_MAS=="YES") & (enriched_points.AGENCY.isna())]) == 0
assert len(enriched_lines[(enriched_lines.COUNTS_TO_MAS=="YES") & (enriched_lines.AGENCY.isna())]) == 0
assert len(enriched_polygons[(enriched_polygons.COUNTS_TO_MAS=="YES") & (enriched_polygons.AGENCY.isna())]) == 0

In [147]:
# Region, vegetation type, ownership must be populated 
assert len(enriched_points[enriched_points.REGION.isna()]) == 0
assert len(enriched_lines[enriched_lines.REGION.isna()]) == 0
assert len(enriched_polygons[enriched_polygons.REGION.isna()]) == 0

assert len(enriched_points[enriched_points.BROAD_VEGETATION_TYPE.isna()]) == 0
assert len(enriched_lines[enriched_lines.BROAD_VEGETATION_TYPE.isna()]) == 0
assert len(enriched_polygons[enriched_polygons.BROAD_VEGETATION_TYPE.isna()]) == 0

assert len(enriched_points[enriched_points.PRIMARY_OWNERSHIP_GROUP.isna()]) == 0
assert len(enriched_lines[enriched_lines.PRIMARY_OWNERSHIP_GROUP.isna()]) == 0
assert len(enriched_polygons[enriched_polygons.PRIMARY_OWNERSHIP_GROUP.isna()]) == 0

In [19]:
with open("..\config.yaml", 'r') as stream:
    config_inputs = yaml.safe_load(stream)

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\sky\AppData\Local\Temp\ipykernel_467764\1274618701.py:1: SyntaxWarning: invalid escape sequence '\c'
  with open("..\config.yaml", 'r') as stream:


In [25]:
a_reference_gdb_path = config_inputs['global']['veg_own_gdb']
#veg_layer = gpd.read_file(a_reference_gdb_path, driver="OpenFileGDB", layer='Broad_Vegetation_Types')
start_year = config_inputs['global']['start_year']
end_year = config_inputs['global']['end_year']

In [22]:
from utils.enrich_polygons import enrich_polygons

In [136]:
from utils.enrich_points import enrich_points

In [78]:
# somehow geom not closed
test_geom = list(enriched_polygons[enriched_polygons.REGION.isna()].geometry)[0].geoms[0]
test_geom.is_closed

False

In [90]:
invalid_rows = pd.concat([enriched_polygons[enriched_polygons.REGION.isna()], enriched_polygons[enriched_polygons.BROAD_VEGETATION_TYPE.isna()]])

In [99]:
enriched_polygons = enriched_polygons.loc[~enriched_polygons.index.isin(invalid_rows.index)]

In [110]:
enriched_points['VALID_GEOM'] = 'VALID'
enriched_lines['VALID_GEOM'] = 'VALID'

In [106]:
enriched_polygons['VALID_GEOM'] = 'VALID'

C:\Users\sky\.conda\envs\its_recode\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [108]:
enriched_polygons.loc[~enriched_polygons.is_valid, 'VALID_GEOM'] = 'FIXED'

In [111]:
repaired_polygons = enriched_polygons[~enriched_polygons.is_valid].make_valid()

In [127]:
# take multipolygon/polygon type, reject geometric collection type
fixed_geoms = repaired_polygons.geom_type.isin(['MultiPolygon', 'Polygon'])
enriched_polygons.loc[repaired_polygons[fixed_geoms].index, 'geometry'] = repaired_polygons[fixed_geoms]

In [128]:
repaired_polygons[~repaired_polygons.geom_type.isin(['MultiPolygon', 'Polygon'])]

GeoSeries([], dtype: geometry)

In [129]:
invalid_rows['VALID_GEOM'] = 'INVALID'

In [134]:
invalid_rows.geometry = invalid_rows.geometry.centroid

In [139]:
out = enrich_points(invalid_rows, a_reference_gdb_path, start_year, end_year)

2025-10-27 14:12:50,047 INFO  [utils.enrich_points   ]        Executing Point Enrichments...
2025-10-27 14:12:50,048 INFO  [utils.enrich_points   ]           Calculating WUI...
2025-10-27 14:12:50,048 INFO  [utils.enrich_points   ]              enrich step 1/16 loading WUI from cache
2025-10-27 14:12:50,208 INFO  [utils.enrich_points   ]                 time for loading WUI: 0.15933823585510254
2025-10-27 14:12:50,208 INFO  [utils.enrich_points   ]              enrich step 2/16 select records with null WUI
2025-10-27 14:12:50,210 INFO  [utils.enrich_points   ]              enrich step 3/16 select by WUI location
2025-10-27 14:12:50,268 INFO  [utils.enrich_points   ]              enrich step 4/16 calculate WUI yes
2025-10-27 14:12:50,269 INFO  [utils.enrich_points   ]              enrich step 5/16 select remaining null records
2025-10-27 14:12:50,270 INFO  [utils.enrich_points   ]              enrich step 6/16 calculate WUI no
2025-10-27 14:12:50,270 INFO  [utils.enrich_points   ]      

In [143]:
enrich_points = pd.concat([enriched_points, out])

In [148]:
# only CNRA is allowed to have ACTIVE status
assert len(enriched_points[(enriched_points.COUNTS_TO_MAS == 'YES') & (enriched_points.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()) == 1
assert enriched_points[(enriched_points.COUNTS_TO_MAS == 'YES') & (enriched_points.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()[0] == 'CNRA'
assert len(enriched_lines[(enriched_lines.COUNTS_TO_MAS == 'YES') & (enriched_lines.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()) == 1
assert enriched_lines[(enriched_lines.COUNTS_TO_MAS == 'YES') & (enriched_lines.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()[0] == 'CNRA'
assert len(enriched_polygons[(enriched_polygons.COUNTS_TO_MAS == 'YES') & (enriched_polygons.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()) == 1
assert enriched_polygons[(enriched_polygons.COUNTS_TO_MAS == 'YES') & (enriched_polygons.ACTIVITY_STATUS == 'ACTIVE')].AGENCY.unique()[0] == 'CNRA'

In [149]:
# Activity category must be populated 
assert len(enriched_lines[enriched_lines.ACTIVITY_CAT.isna()]) == 0
assert len(enriched_points[enriched_points.ACTIVITY_CAT.isna()]) == 0
assert len(enriched_polygons[enriched_polygons.ACTIVITY_CAT.isna()]) == 0
# Activity category with NOT_DEFINED can only come from activity description of TBD or NOT_DEFINED
assert set(enriched_points[enriched_points.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()).issubset(set(['TBD', 'NOT_DEFINED']))
assert set(enriched_lines[enriched_lines.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()).issubset(set(['TBD', 'NOT_DEFINED']))
assert set(enriched_polygons[enriched_polygons.ACTIVITY_CAT == 'NOT_DEFINED'].ACTIVITY_DESCRIPTION.unique()).issubset(set(['TBD', 'NOT_DEFINED']))

In [150]:
# check for valid subset of agency
enriched_polygons.AGENCY.unique()
enriched_points.AGENCY.unique()
enriched_lines.AGENCY.unique()

array(['CALSTA', 'CNRA'], dtype=object)

In [151]:
# Threshold of na %
cols =["AGENCY",
       "ADMINISTERING_ORG",
       "PRIMARY_OWNERSHIP_GROUP",
       "COUNTY",
       "REGION",
       "ACTIVITY_DESCRIPTION",
       "ACTIVITY_CAT",
       "BROAD_VEGETATION_TYPE",
       "ACTIVITY_STATUS",
       "ACTIVITY_QUANTITY",
       "ACTIVITY_UOM",
       "ACTIVITY_END",
       "Year_txt"]
# change this value accordingly
NA_THRESH = 0.01
for col in cols:
    assert sum(enriched_lines[col].isna())/len(enriched_lines) < NA_THRESH
    assert sum(enriched_points[col].isna())/len(enriched_points) < NA_THRESH
    assert sum(enriched_polygons[col].isna())/len(enriched_polygons) < NA_THRESH

In [155]:
save_gdf_to_gdb(enriched_polygons, fpath, 'appended_poly')
save_gdf_to_gdb(enriched_points, fpath, 'appended_point')
save_gdf_to_gdb(enriched_lines, fpath, 'appended_line')

2025-10-27 14:17:48,717 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-10-27 14:17:48,717 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-10-27 14:17:48,800 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb


2025-10-27 14:18:25,113 INFO  [pyogrio._io           ]  Created 457,710 records
2025-10-27 14:18:33,913 INFO  [utils.save_gdf_to_gdb ]        File saved to D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb appended_poly
2025-10-27 14:18:33,914 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-10-27 14:18:33,914 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-10-27 14:18:33,917 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb


2025-10-27 14:18:35,107 INFO  [pyogrio._io           ]  Created 20,505 records
2025-10-27 14:18:35,281 INFO  [utils.save_gdf_to_gdb ]        File saved to D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb appended_point
2025-10-27 14:18:35,281 INFO  [utils.save_gdf_to_gdb ]        Windows machine detected
2025-10-27 14:18:35,282 INFO  [utils.save_gdf_to_gdb ]        Check geodataframe geometry object and cast to Multi-x type if both exist
2025-10-27 14:18:35,294 INFO  [utils.save_gdf_to_gdb ]        Running GDAL OpenFileGDB to save to file


D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb


2025-10-27 14:19:02,445 INFO  [pyogrio._io           ]  Created 131,744 records
2025-10-27 14:19:08,089 INFO  [utils.save_gdf_to_gdb ]        File saved to D:/WORK/wildfire/Interagency-Tracking-System/its/ITSGDB_backup/V2.0.2/appended.gdb appended_line
